# Ingenierías de Características

# Línea base del modelo

## Introducción

En este tutorial, aprenderá un enfoque práctico para la ingeniería de características. A través de ejercicios prácticos podrás:

+ Desarrollar un modelo de referencia (*baseline*) para comparar el rendimiento en modelos con más características.
+ Codificar características categóricas para que el modelo pueda hacer un mejor uso de la información.
+ Generar nuevas funciones para proporcionar más información para el modelo.
+ Seleccionar funciones para reducir el sobreajuste y aumentar la velocidad de predicción.

En el ejercicio, aplicará técnicas de ingeniería de características a los datos de la competición [TalkingData AdTracking](https://www.kaggle.com/c/talkingdata-adtracking-fraud-detection).

![TalkingData](https://i.imgur.com/srKxEkD.png)

El objetivo de la competición es predecir si un usuario descargará una aplicación después de hacer clic en un anuncio. Para este curso, utilizará una pequeña muestra de los datos, eliminando el 99% de los registros negativos (donde la aplicación no se descargó) para hacer que el objetivo sea más equilibrado.

### Proyectos Kickstarter

Para ayudarle a aplicar las técnicas, primero las verá aplicadas utilizando datos de proyectos de Kickstarter. Las primeras filas de los datos de los proyectos de Kickstarter se ven así:

In [1]:
import pandas as pd
ks = pd.read_csv('./input/kickstarter-projects/ks-projects-201801.csv',
                 parse_dates=['deadline', 'launched'])
ks.head(10)

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.00,failed,0,GB,0.00,0.00,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.00,failed,15,US,100.00,2421.00,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.00,failed,3,US,220.00,220.00,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.00,failed,1,US,1.00,1.00,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.00,canceled,14,US,1283.00,1283.00,19500.00
5,1000014025,Monarch Espresso Bar,Restaurants,Food,USD,2016-04-01,50000.0,2016-02-26 13:38:27,52375.00,successful,224,US,52375.00,52375.00,50000.00
6,1000023410,Support Solar Roasted Coffee & Green Energy! ...,Food,Food,USD,2014-12-21,1000.0,2014-12-01 18:30:44,1205.00,successful,16,US,1205.00,1205.00,1000.00
7,1000030581,Chaser Strips. Our Strips make Shots their B*tch!,Drinks,Food,USD,2016-03-17,25000.0,2016-02-01 20:05:12,453.00,failed,40,US,453.00,453.00,25000.00
8,1000034518,SPIN - Premium Retractable In-Ear Headphones w...,Product Design,Design,USD,2014-05-29,125000.0,2014-04-24 18:14:43,8233.00,canceled,58,US,8233.00,8233.00,125000.00
9,100004195,STUDIO IN THE SKY - A Documentary Feature Film...,Documentary,Film & Video,USD,2014-08-10,65000.0,2014-07-11 21:55:48,6240.57,canceled,43,US,6240.57,6240.57,65000.00


Lo que podemos hacer aquí es predecir si un proyecto de Kickstarter tendrá éxito. Obtenemos el resultado de la columna de `estado`. Para predecir el resultado, podemos usar características como categoría, moneda, objetivo de financiación, país y cuándo se lanzó.

### Preparando columna objetivo

Primero miraré los estados del proyecto y convertiré la columna en algo que podamos usar como objetivos en un modelo.

In [2]:
pd.unique(ks.state)

array(['failed', 'canceled', 'successful', 'live', 'undefined',
       'suspended'], dtype=object)

Tenemos seis estados, ¿Cuántos registros cada uno?

In [3]:
ks.groupby("state")["ID"].count()

state
canceled       38779
failed        197719
live            2799
successful    133956
suspended       1846
undefined       3562
Name: ID, dtype: int64

La limpieza de datos no es el enfoque actual, por lo que simplificaremos este ejemplo:

+ Descartando de proyectos que están "en vivo"
+ Contando estados "exitosos" como `outcome = 1`
+ Combinando cualquier otro estado como `outcome = 0`

In [4]:
# Elimina los proyectos vivos
ks = ks.query('state != "live"')

# Añade columna de resultado, "successful" == 1, en cualquier otro caso es 0
ks = ks.assign(outcome=(ks['state'] == 'successful').astype(int))

In [9]:
ks.sample(5)

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real,outcome
229290,235772762,Mutation: Affected and defective,Documentary,Film & Video,USD,2015-10-28,130000.0,2015-09-29 02:39:48,0.0,failed,0,US,0.00,0.00,130000.00,0
247720,329955578,Simien Mountains photography book,Art Books,Publishing,GBP,2015-06-12,16500.0,2015-05-13 12:07:06,17738.0,successful,329,GB,27629.33,27470.11,25552.87,1
149688,1760685207,Alexander Jansson 2016 Art Calendar,Digital Art,Art,SEK,2015-09-30,30000.0,2015-09-15 12:22:54,200455.0,successful,418,SE,24370.05,23948.37,3584.10,1
40607,1206658616,Adoris Samuels: A New Fashion Label Is Born,Apparel,Fashion,USD,2014-08-19,7000.0,2014-07-15 14:17:53,175.0,failed,2,US,175.00,175.00,7000.00,0
293244,562673359,Darth Vader captured a place for rest.,Shorts,Film & Video,EUR,2016-06-22,1600.0,2016-05-23 13:27:25,1.0,failed,1,DE,1.12,1.11,1770.56,0


### Convertir timestamps

Convierto la característica `launched` en categórica que podemos usarla en un modelo. Como cargué la columna como de tipo fecha, accedo a los valores de fecha y hora a través del atributo .dt.

In [10]:
ks = ks.assign(hour=ks.launched.dt.hour,
               day=ks.launched.dt.day,
               month=ks.launched.dt.month,
               year=ks.launched.dt.year)

ks.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real,outcome,hour,day,month,year
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95,0,12,11,8,2015
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00,0,4,2,9,2017
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00,0,0,12,1,2013
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00,0,3,17,3,2012
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00,0,8,4,7,2015


### Preparación de variables categóricas

Ahora, para las variables categóricas (`category`, `currency` y `country`), tendré que convertirlas en números enteros para que nuestro modelo pueda usar los datos. Para esto usaré el `LabelEncoder` de scikit-learn. Esto asigna un número entero a cada valor de la característica categórica y reemplaza esos valores con los números enteros.

In [11]:
from sklearn.preprocessing import LabelEncoder

cat_features = ['category', 'currency', 'country']
encoder = LabelEncoder()

# Aplica la codificacion de etiqueta a cada columna
encoded = ks[cat_features].apply(encoder.fit_transform)
encoded.head(10)

,category,currency,country
0,108,5,9
1,93,13,22
2,93,13,22
3,90,13,22
4,55,13,22
5,123,13,22
6,58,13,22
7,41,13,22
8,113,13,22
9,39,13,22


Recopilaré todas las características que usaremos en un nuevo dataframe y lo usaré para entrenar un modelo.

In [12]:
# Dado que ks y encoded tienen el mismo índice puedo unirlos fácilmente
data = ks[['goal', 'hour', 'day', 'month', 'year', 'outcome']].join(encoded)
data.head()

,goal,hour,day,month,year,outcome,category,currency,country
0,1000.0,12,11,8,2015,0,108,5,9
1,30000.0,4,2,9,2017,0,93,13,22
2,45000.0,0,12,1,2013,0,93,13,22
3,5000.0,3,17,3,2012,0,90,13,22
4,19500.0,8,4,7,2015,0,55,13,22


### Creación divisiones de entrenamiento, validación y prueba

Necesitamos crear conjuntos de datos para entrenamiento, validación y pruebas. Usaremos un enfoque bastante simple y dividiremos los datos usando segmentos. Utilizaremos el 10% de los datos como un conjunto de validación, el 10% para las pruebas y el otro 80% para el entrenamiento.

In [13]:
valid_fraction = 0.1
valid_size = int(len(data) * valid_fraction)

train = data[:-2 * valid_size]
valid = data[-2 * valid_size:-valid_size]
test = data[-valid_size:]

En general, debe tener cuidado de que cada conjunto de datos tenga la misma proporción de clases objetivo. Mostraré la fracción de resultados exitosos para cada uno de nuestros conjuntos de datos.

In [14]:
for each in [train, valid, test]:
    print(f"Fracción de resultados = {each.outcome.mean():.4f}")

Fracción de resultados = 0.3570
Fracción de resultados = 0.3539
Fracción de resultados = 0.3542


Esto se ve bien, cada conjunto tiene alrededor del 35% de resultados verdaderos, probablemente porque los datos estaban bien aleatorios de antemano. Una buena manera de hacer esto automáticamente es con `sklearn.model_selection.StratifiedShuffleSplit` pero no necesito usarlo aquí.

### Entrenando un modelo LightGBM

Para este curso usaremos un modelo LightGBM. Este es un modelo basado en árbol que generalmente proporciona el mejor rendimiento, incluso en comparación con XGBoost. También es relativamente rápido de entrenar. No haremos la optimización de hiperparámetros porque ese no es el objetivo de este curso. Por lo tanto, nuestros modelos no tendrán el mejor rendimiento absoluto que puedan obtener. Pero aún así verá mejorar el rendimiento del modelo a medida que hacemos ingeniería de características.

In [16]:
import lightgbm as lgb

feature_cols = train.columns.drop('outcome')

dtrain = lgb.Dataset(train[feature_cols], label=train['outcome'])
dvalid = lgb.Dataset(valid[feature_cols], label=valid['outcome'])

param = {'num_leaves': 64, 'objective': 'binary'}
param['metric'] = 'auc'
num_round = 1000
bst = lgb.train(param, dtrain, num_round, valid_sets=[dvalid], early_stopping_rounds=10, verbose_eval=False)

### Hacer predicciones y evaluar el modelo

Finalmente, hagamos predicciones sobre el conjunto de pruebas con el modelo y veamos cómo de bien funciona. Una cosa importante para recordar es que puede sobreajustar los datos de validación. Es por eso que necesitamos un conjunto de prueba que el modelo nunca ve hasta la evaluación final.

In [17]:
from sklearn import metrics
ypred = bst.predict(test[feature_cols])
score = metrics.roc_auc_score(test['outcome'], ypred)

print(f"Test AUC score: {score}")

Test AUC score: 0.747615303004287


## Ejercicio

En este ejercicio, desarrollará un modelo de referencia para predecir si un cliente comprará una aplicación después de hacer clic en un anuncio. Con este modelo de referencia, podrá ver cómo su ingeniería de características y esfuerzos de selección mejoran el rendimiento del modelo.

In [18]:
import pandas as pd

click_data = pd.read_csv('./input/feature-engineering-data/train_sample.csv',
                         parse_dates=['click_time'])
click_data.head(10)

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
0,87540,12,1,13,497,2017-11-07 09:30:38,NaN,0
1,105560,25,1,17,259,2017-11-07 13:40:27,NaN,0
2,101424,12,1,19,212,2017-11-07 18:05:24,NaN,0
3,94584,13,1,13,477,2017-11-07 04:58:08,NaN,0
4,68413,12,1,1,178,2017-11-09 09:00:09,NaN,0
5,93663,3,1,17,115,2017-11-09 01:22:13,NaN,0
6,17059,1,1,17,135,2017-11-09 01:17:58,NaN,0
7,121505,9,1,25,442,2017-11-07 10:01:53,NaN,0
8,192967,2,2,22,364,2017-11-08 09:35:17,NaN,0
9,143636,3,1,19,135,2017-11-08 12:35:26,NaN,0


### Linea base del modelo

Lo primero que debe hacer es construir una línea base del modelo. Todas las características nuevas, el procesamiento, las codificaciones y la selección de características deberían mejorar esta línea base. Primero debe hacer un poco de ingeniería de características antes de entrenar el modelo en sí.

#### 1) Características de los timestamps
A partir de los timestamps, cree funciones para el día, hora, minuto y segundo. Almacénelos como nuevas columnas enteras `day`,` hour`, `minute` y` second` en un nuevo DataFrame `clicks`.

In [19]:
# Añade nuevas columnas para las características timestamps day, hour, minute, y second
clicks = click_data.copy()
clicks['day'] = clicks['click_time'].dt.day.astype('uint8')
clicks['hour'] = clicks['click_time'].dt.hour.astype('uint8')
clicks['minute'] = clicks['click_time'].dt.minute.astype('uint8')
clicks['second'] = clicks['click_time'].dt.second.astype('uint8')

In [20]:
clicks.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed,day,hour,minute,second
0,87540,12,1,13,497,2017-11-07 09:30:38,NaN,0,7,9,30,38
1,105560,25,1,17,259,2017-11-07 13:40:27,NaN,0,7,13,40,27
2,101424,12,1,19,212,2017-11-07 18:05:24,NaN,0,7,18,5,24
3,94584,13,1,13,477,2017-11-07 04:58:08,NaN,0,7,4,58,8
4,68413,12,1,1,178,2017-11-09 09:00:09,NaN,0,9,9,0,9


#### 2) Codificación de etiquetas

Para cada una de las funciones categóricas `['ip', 'app', 'device', 'os', 'channel']`, use el `LabelEncoder` de scikit-learn para crear nuevas funciones en el Dataframe `clicks`. Los nuevos nombres de columna deben ser el nombre original de la columna con `'_labels'` añadido, como `ip_labels`.

In [21]:
from sklearn import preprocessing

cat_features = ['ip', 'app', 'device', 'os', 'channel']
label_encoder = preprocessing.LabelEncoder()

# Crea nuevas columnas usando preprocessing.LabelEncoder()
for feature in cat_features:
    encoded = label_encoder.fit_transform(clicks[feature])
    clicks[feature + "_labels"] = encoded

In [22]:
clicks.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed,day,hour,minute,second,ip_labels,app_labels,device_labels,os_labels,channel_labels
0,87540,12,1,13,497,2017-11-07 09:30:38,NaN,0,7,9,30,38,15220,11,1,13,159
1,105560,25,1,17,259,2017-11-07 13:40:27,NaN,0,7,13,40,27,18448,24,1,17,67
2,101424,12,1,19,212,2017-11-07 18:05:24,NaN,0,7,18,5,24,17663,11,1,19,52
3,94584,13,1,13,477,2017-11-07 04:58:08,NaN,0,7,4,58,8,16496,12,1,13,146
4,68413,12,1,1,178,2017-11-09 09:00:09,NaN,0,9,9,0,9,11852,11,1,1,45


#### 3) Codificación One-hot

Ahora que tiene características *label encoded*, ¿tiene sentido usar la codificación one-hot para las variables categóricas ip, aplicación, dispositivo, sistema operativo o canal?

**Solución**: La columna `ip` tiene 58.000 valores, lo que significa que creará una matriz dispersa con 58.000 columnas. Esta cantidad de columnas hará que su modelo funcione muy lentamente, por lo que, en general, deseará evitar las codificaciones one-hot de características con muchos niveles. Los modelos LightGBM funcionan con características *label encoded*, por lo que en realidad no necesita codificar one-hot las características categóricas.

### Conjuntos de entrenamiento, validación y prueba

Con nuestras características de línea base listas, necesitamos dividir nuestros datos en conjuntos de entrenamiento y validación. También deberíamos ofrecer un conjunto de pruebas para medir la precisión final del modelo.

#### 4) División de entrenamientos/pruebas con datos de series de tiempo

Estos son datos de series de tiempo. ¿Hay alguna consideración especial al crear divisiones de entrenamiento/prueba para series de tiempo? ¿Si es así, qué y por qué?

**Solución**: Dado que nuestro modelo está destinado a predecir eventos en el futuro, también debemos validar el modelo en eventos en el futuro. Si los datos se mezclan entre los conjuntos de entrenamiento y prueba, los datos futuros se *fugarán* al modelo y nuestros resultados de validación sobreestimarán el rendimiento en los nuevos datos.

#### Crear divisiones de entrenamiento/validación/prueba

Aquí crearemos divisiones de entrenamiento, validación y prueba. Primero, el dataframe `clicks` se ordena en orden de tiempo creciente. El primer 80% de las filas son el conjunto de entrenamiento, el siguiente 10% son el conjunto de validación y el último 10% son el conjunto de prueba.

In [23]:
feature_cols = ['day', 'hour', 'minute', 'second', 
                'ip_labels', 'app_labels', 'device_labels',
                'os_labels', 'channel_labels']

valid_fraction = 0.1
clicks_srt = clicks.sort_values('click_time')
valid_rows = int(len(clicks_srt) * valid_fraction)
train = clicks_srt[:-valid_rows * 2]
# valid size == test size, last two sections of the data
valid = clicks_srt[-valid_rows * 2:-valid_rows]
test = clicks_srt[-valid_rows:]

#### Entrenar con LightGBM

Ahora podemos crear objetos de conjunto de datos LightGBM para cada uno de los conjuntos de datos más pequeños y entrenar el modelo de línea base.

In [28]:
import lightgbm as lgb

dtrain = lgb.Dataset(train[feature_cols], label=train['is_attributed'])
dvalid = lgb.Dataset(valid[feature_cols], label=valid['is_attributed'])
dtest = lgb.Dataset(test[feature_cols], label=test['is_attributed'])

param = {'num_leaves': 64, 'objective': 'binary'}
param['metric'] = 'auc'
num_round = 1000
bst = lgb.train(param, dtrain, num_round, valid_sets=[dvalid], early_stopping_rounds=10)

[1]	valid_0's auc: 0.932413
Training until validation scores don't improve for 10 rounds
[2]	valid_0's auc: 0.530091
[3]	valid_0's auc: 0.446663
[4]	valid_0's auc: 0.608956
[5]	valid_0's auc: 0.608624
[6]	valid_0's auc: 0.809223
[7]	valid_0's auc: 0.810089
[8]	valid_0's auc: 0.810314
[9]	valid_0's auc: 0.80959
[10]	valid_0's auc: 0.809999
[11]	valid_0's auc: 0.809321
Early stopping, best iteration is:
[1]	valid_0's auc: 0.932413


# ## Evaluar el modelo

Finalmente, con el modelo entrenado, evaluaré su rendimiento en el conjunto de pruebas.

In [26]:
from sklearn import metrics

ypred = bst.predict(test[feature_cols])
score = metrics.roc_auc_score(test['is_attributed'], ypred)
print(f"Test score: {score}")

Test score: 0.7811898797595189


Esta será nuestra puntuación de línea base del modelo. Cuando transformamos características, agregamos nuevas o realizamos una selección de características, deberíamos estar mejorando en esta puntuación. Sin embargo, dado que este es el conjunto de prueba, solo queremos verlo al final de todas nuestras manipulaciones. Al final de este curso, volverá a ver la puntuación de la prueba para ver si mejoró la línea base del modelo.